# 1.Import Data

In [1]:
import numpy as np
import pickle
import sklearn
from tqdm import tqdm

In [2]:
y_name={
    "y_train":"train_y",
    "y_test":"test_y"
}

In [3]:
def open_pkl(file_name):
    all_data=dict()
    for each_key in file_name.keys():
        name = file_name[each_key]
        pickle_file = open('DataSet\\'+name+".pkl",mode="rb")
        data = pickle.load(pickle_file)
        pickle_file.close()
        all_data[each_key]=data
    return all_data

In [4]:
y_data = open_pkl(y_name)
y_train = y_data["y_train"]
y_test = y_data["y_test"]

In [5]:
y_train

array([0, 3, 0, ..., 0, 0, 0])

In [6]:
file_name0={
    "3turn":"vec_train_3_no_cor_word2vec_change",
    }
file_name1={
    "3turn":"vec_test_3_no_cor_word2vec_change",
}

In [7]:
#import data
def open_pkl_data(file_name):
    all_data=dict()
    for each_key in file_name.keys():
        name = file_name[each_key]
        pickle_file = open('DataSet\\'+name+".pkl",mode="rb")
        data = pickle.load(pickle_file)
        pickle_file.close()
        all_data[each_key]=data
    return all_data

In [8]:
train_data_overall = open_pkl_data(file_name0)
test_data_overall = open_pkl_data(file_name1)

In [9]:
original_train = train_data_overall["3turn"]
original_test = test_data_overall["3turn"]

# 2. Calculation of the average vectors of each sentence

In [10]:
length = len(original_train["overall"][0][0])
print(length)

300


In [11]:
def cal_average(all_data):
    output_dict=dict()
    for key in all_data.keys():
        sentences = list()
        for sentence in all_data[key]:
            sentence = np.array(sentence)
            length = len(sentence)
            sum_sen = sentence.sum(axis=0)
            aver_sen = sum_sen/length
            sentences.append(aver_sen)
        output_dict[key]=sentences
    
    return output_dict

In [12]:
ave_train = cal_average(original_train)
ave_test = cal_average(original_test)

In [13]:
length = len(ave_train["overall"][0])
print(length)

300


In [14]:
len(ave_train["overall"])

30160

# 3.Logistic Regression Model
## 3.1. build and fit model

In [15]:
from sklearn.linear_model import LogisticRegression
def lr_model(train_data,y_train):
    classifiers = dict()
    for key in tqdm(train_data.keys()):
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(train_data[key],y_train)
        classifiers[key]=classifier
    
    return classifiers

In [16]:
classifiers = lr_model(ave_train,y_train)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
 33%|████████████████████████████                                                        | 1/3 [00:02<00:05,  2.58s/it]C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data 

In [17]:
# save model
def save_data(name,data):
    pickle_file = open('DataSet\\'+name+".pkl",mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [18]:
# save model
save_data("LogisticRegression_comparison_word2vec",classifiers)

## 3.2. prediction

In [19]:
# function for getting prediction
def get_prediction(classifiers, ave_test):
    y_pred_dict= dict()
    list1 = list(classifiers.keys())
    list2 = list(ave_test.keys())
    length = len(list1)
    
    for i in tqdm(range(length)):
        y_pred_dict[list2[i]]=classifiers[list1[i]].predict(ave_test[list2[i]])
        
    return y_pred_dict

In [20]:
y_preds = get_prediction(classifiers,ave_test)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 45.58it/s]


In [21]:
# function for getting the score of each prediction
def get_scores(classifiers, ave_test):
    y_score_dict= dict()
    list1 = list(classifiers.keys())
    list2 = list(ave_test.keys())
    length = len(list1)
    
    for i in tqdm(range(length)):
        y_score_dict[list2[i]]=classifiers[list1[i]].decision_function(ave_test[list2[i]])
        
    return y_score_dict

In [22]:
y_scores = get_scores(classifiers, ave_test)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 41.21it/s]


In [23]:
# function for saving the predictions and scores
def save_prediction(name,data):
    pickle_file = open('Prediction\\'+name+".pkl",mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [24]:
save_prediction("LogisticRegression_process3turn_word2vec_preds",y_preds)
save_prediction("LogisticRegression_process3turn_word2vec_scores",y_scores)

## 3.3. Evaluation: confusion matrix, accuracy, precision, recall, F1 measure

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# print evaluation result
def get_evaluation(y_test,y_pred):
    ev = dict()
    ev["confusion_matrix"]=confusion_matrix(y_test,y_pred)
    ev["accuracy"] = accuracy_score(y_test,y_pred)
    ev["precision_macro"]=precision_score(y_test,y_pred,average="macro")
    ev["precision_micro"]=precision_score(y_test,y_pred,average="micro")
    ev["precision_weighted"]=precision_score(y_test,y_pred,average="weighted")
    ev["recall_macro"]=recall_score(y_test,y_pred,average="macro")
    ev["recall_micro"]=recall_score(y_test,y_pred,average="micro")
    ev["recall_weighted"]=recall_score(y_test,y_pred,average="weighted")
    ev["F1_score_macro"]=f1_score(y_test,y_pred,average="macro")
    ev["F1_score_micro"]=f1_score(y_test,y_pred,average="micro")
    ev["F1_score_weighted"]=f1_score(y_test,y_pred,average="weighted")
    
    for key in ev.keys():
        if key !="confusion_matrix":
            print("{a} is: {b}".format(a=key, b=ev[key]))
        else:
            print(ev[key])
    
    return ev

    

In [26]:
#"macro": "Calculate metrics for each label, and find their unweighted mean. ",
#"micro": "Calculate metrics globally by counting the total true positives, false negatives and false positives."
#"weighted":"Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). "

def get_evaluation_all(y_test, y_pred_dict):
    for key in tqdm(y_pred_dict.keys()):
        print("evaluation on data {0}".format(key))
        get_evaluation(y_test, y_pred_dict[key])
        print("***************************************")
    

In [27]:
get_evaluation_all(y_test,y_preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 34.18it/s]

evaluation on data overall
[[4018  183  241  235]
 [ 149  113    9   13]
 [  90    5  140   15]
 [  87   12   12  187]]
accuracy is: 0.8092212742784535
precision_macro is: 0.512447646431204
precision_micro is: 0.8092212742784535
precision_weighted is: 0.84215651699373
recall_macro is: 0.6111254536689582
recall_micro is: 0.8092212742784535
recall_weighted is: 0.8092212742784535
F1_score_macro is: 0.5497044120490444
F1_score_micro is: 0.8092212742784535
F1_score_weighted is: 0.8223257889178816
***************************************
evaluation on data double13
[[4028  179  244  226]
 [ 131  127   16   10]
 [  77    5  150   18]
 [  70    7   14  207]]
accuracy is: 0.8190234162279906
precision_macro is: 0.5344018594233075
precision_micro is: 0.8190234162279906
precision_weighted is: 0.8550956127656971
recall_macro is: 0.650762451502924
recall_micro is: 0.8190234162279906
recall_weighted is: 0.8190234162279906
F1_score_macro is: 0.5773225973938658
F1_score_micro is: 0.8190234162279906
F1_s

# 4. if just want to directly load the model

In [28]:
model_names = {
    "model_all":"LogisticRegression_comparison_word2vec"
}

In [29]:
model_data = open_pkl(model_names)
classifiers = model_data